# Het stress-dossier van Keuzegids


<img align="right" width="100" height="100" src="https://www.keuzegids.org/ol/gidsen/hbovt20/img/layout/logo.jpg">




Op 4 november 2019 publiceerde Keuzegids een [persbericht](https://keuzegids.nl/gestreste-student-haalt-gewoon-diploma/) waarin het stelde dat studenten aan de HvA en HL meer stress ervoeren dan studenten bij andere instellingen.
Hieronder worden de cijfers zoals Keuzegids die op verzoek toestuurde gereproduceerd. Wat zijn dat eigenlijk voor cijfers? Waar komen de uitspraken over stress en studiesucces vandaan? 

## Waar gaat het over?
Keuzegids heeft dit jaar een nieuw thema in het leven geroepen, 'haalbaarheid'. Het thema bestaat uit een aantal items, afkomstig uit een aantal andere thema's van de NSE. 

## Wat is haalbaarheid?
Hieronder staan de items waaruit het thema 'haalbaarheid' is opgebouwd:

1. De spreiding van de studielast over het studiejaar (thema studielast, Studielast_01)
1. De haalbaarheid van deadlines (thema studielast, Studielast_02)
1. De mogelijkheid om zonder vertraging de gewenste studie-onderdelen te volgen (thema studielast, Studielast_04)
1. De studeerbaarheid van het studierooster (bijv. spreiding en tijdstippen) (thema studierooster, Studierooster_03)
1. De informatie over jouw studievoortgang (thema informatievoorziening, Informatievoorziening_02)
1. Het tijdig bekend maken van resultaten van toetsen en beoordelingen (thema informatievoorziening, Informatievoorziening_05)

## Hoe wordt de stress-barometer gemaakt?
Per instelling x opleiding wordt de score op het thema haalbaarheid berekend. Vervolgens wordt het gemiddelde van alle opleidingsscores berekend, zodat er een score per instelling ontstaat. Alleen instellingen met meer dan 15 opleidingen worden opgenomen in de ranglijst.

## Aanpak
Het maken van de ranglijst wordt opgeknipt in een paar stappen:

1. ophalen en prepareren data
1. toevoegen themascore 'haalbaarheid'
1. filter op relevante records
1. berekenen scores per instelling x opleiding
1. berekenen scores per instelling

## 1. ophalen en prepareren data

In [ ]:
# importeer modules die we nodig hebben
import altair as alt
from altair import datum
import pandas as pd
import numpy as np
import requests

# een variabele met alle G5-brins:
g5_brin = ['21RI','27UM', '22OJ' ,'28DN', '27PZ','25DW']

In [ ]:
url = nse_data_url

# haal de data op van een url:
r = requests.get(url)
# schrijf de data weg in een bestand op je harde schijf met de bestandsnaam hieronder:
with open('nse_data.csv.zip', 'wb') as f:
    f.write(r.content)

In [ ]:
# Lees de data in een tabel:
nse_data=pd.read_csv('nse_data.csv.zip')

### Hoe ziet de data eruit? Klopt die een beetje?

In [ ]:
len(nse_data)

In [ ]:
nse_data.head()

## 2. toevoegen themascore 'haalbaarheid'

Per student wordt hieronder de themascore 'haalbaarheid' berekend. Dat is niet meer dan het gemiddelde van de scores op de items die horen bij de schaal haalbaarheid.

In [ ]:
# De items die tot de schaal horen in een list:

haalbaarheid = [ 'Studielast_01'
               , 'Studielast_02'
               , 'Studielast_04'
               , 'Studierooster_03'
               , 'Informatievoorziening_02'
               , 'Informatievoorziening_05']

# Maak een themascore haalbaarheid:
nse_data['Haalbaarheid'] = np.mean(nse_data[haalbaarheid], axis=1)

## 3. filter relevante records
We willen alleen oordelen van opleidingen die voldoen aan de volgende filters:
1. Voltijd
1. HBO
1. Bachelor
1. 2018
1. Bekostigd
1. Instellingen met meer dan 15 opleidingen

In [ ]:
# Hieronder een functie die, als je er data aan geeft, 
# een lijst met brin nummers geeft die meer dan 15 opleidingen hebben.

def get_brin_instellingen_met_meer_dan_15_opleidingen(data):
    """Lijst met brin-nummers van instellingen met meer dan
    15 opleidingen"""
    groupers = ['BrinActueel','BrinNaamActueel','Locatie']
    aantal_opl_inst_vestiging = data.groupby(groupers).CrohoNaamActueel.nunique().rename('Aantal').reset_index()
    aantal_opl_inst = aantal_opl_inst_vestiging.groupby(['BrinActueel','BrinNaamActueel']).Aantal.sum().reset_index()
    brin_instellingen_meer_dan_15 = aantal_opl_inst[aantal_opl_inst.Aantal >= 15].BrinActueel.tolist()
    
    return brin_instellingen_meer_dan_15

In [ ]:
def get_hbo_vt_2018(data):
    """Filtert data van hbo, vt, bachelors 2018, instellingen met meer dan 15 opleidingen."""
    
    brin_nummers = get_brin_instellingen_met_meer_dan_15_opleidingen(nse_data)
    
    filter1 = data.Opleidingsvorm3 == 'Voltijd'
    filter2 = data.SoortHo == 'HBO'
    filter3 = data.BaMa == 'Bachelor'
    filter4 = data.Jaar == 2018
    filter5 = data.Bekostiging=='Bekostigd'
    filter6 = data.BrinActueel.isin(brin_nummers)

    hbo_vt_ba = data[(filter1) & (filter2) & (filter3) & (filter4) & (filter5) & (filter6)].copy()
    
    return hbo_vt_ba

hbo_vt_2018 = get_hbo_vt_2018(nse_data)

In [ ]:
def get_hsleiden_hbo_vt_2018(data):
    """Filtert op Hsleiden"""

    hsleiden_ruw = data[(data.BrinActueel=='21RI')].copy()
    
    return hsleiden_ruw

hsleiden2018 = get_hsleiden_hbo_vt_2018(hbo_vt_2018)

## 4. berekenen scores per instelling x opleiding

In [ ]:
groupers = ['BrinNaamActueel','Locatie','CrohoNaamActueel']

#score_inst_vst_opl = hbo_vt_2018.groupby(groupers).apply(lambda x: indicators(x, columns=['Haalbaarheid',])).reset_index()

In [ ]:
score_inst_vst_opl = hbo_vt_2018.groupby(groupers).Haalbaarheid.mean().rename('Score').reset_index()

## 5. berekenen scores per instelling

In [ ]:
ranglijst = score_inst_vst_opl.groupby(['BrinNaamActueel',]).mean().reset_index().sort_values(by=['Score'], ascending=False).round(decimals=2)

In [ ]:
bars = alt.Chart(ranglijst).mark_bar().encode(
    y=alt.Y('BrinNaamActueel', title=None, sort=alt.EncodingSortField(field='Score'))
    , x='Score'
)

text = alt.Chart(ranglijst).mark_text(dx=14).encode(
    y=alt.Y('BrinNaamActueel', title=None, sort=alt.EncodingSortField(field='Score'))
    , x='Score'
    , text='Score'
)

bars + text

In [ ]:
cols = haalbaarheid + ['Haalbaarheid']

scores_per_instelling = hbo_vt_2018.groupby(['BrinNaamActueel','BrinActueel']).mean()[cols].reset_index()

In [ ]:
long = pd.melt(scores_per_instelling
        , id_vars=['BrinNaamActueel','BrinActueel']
         , var_name='Variabele'
         , value_name='Score')

In [ ]:
filter_g5 = long.BrinActueel.isin(g5_brin)
filter_haalbaarheid=long.Variabele!='Haalbaarheid'
alt.Chart(long[(filter_g5) & (filter_haalbaarheid)],width=180).mark_bar().encode(
      y='Score'
    , x=alt.X('Variabele', sort=alt.EncodingSortField(field='Variabele'))).facet(column='BrinNaamActueel')


## Controle

We kijken op een aantal manieren of de manier waarop Keuzegids de schaal maakt hout snijdt. Hieronder een correlatiematrix van alle items.

### Correlatiematrix

In [ ]:
hbo_vt_2018[haalbaarheid].corr()

## Cronbach's alpha
Keuzegids maakt een eigen schaal. Maar is het ook een schaal? Cronbach's alpha kan ons helpen een indicatie krijgen van de betrouwbaarheid van de schaal/

In [ ]:
import numpy
def cronbach(itemscores):
    itemscores = numpy.asarray(itemscores)
    itemvars = itemscores.var(axis=1, ddof=1)
    tscores = itemscores.sum(axis=0)
    nitems = len(itemscores)

    return nitems / (nitems-1.) * (1 - itemvars.sum() / tscores.var(ddof=1))

In [ ]:
def get_itemscores(data, items):
    """Maakt een lijst van lijsten. Iedere lijst zijn de scores op een item, missings
    worden opgevuld met het gemiddelde."""
    itemscores = []
    data = data[items].dropna()
    for item in items:
        itemscores_filled = data[item].fillna(data[item].mean()).tolist()
        itemscores.append(itemscores_filled)
    return itemscores
    
itemscores = get_itemscores(hbo_vt_2018, haalbaarheid)

In [ ]:
cronbach(itemscores)

Met een cronbach's alpha van 0,81 kan je zeggen dat de schaal betrouwbaarheid is. De items hangen dus wel voldoende met elkaar samen.

### Maar zijn het reeele verschillen?

In [ ]:
from statsmodels.stats.multicomp import pairwise_tukeyhsd
import scipy.stats as stats

tukey = pairwise_tukeyhsd(endog=hbo_vt_2018.dropna(subset=['Haalbaarheid']).Haalbaarheid,     # Data
                          groups=hbo_vt_2018.dropna(subset=['Haalbaarheid'])['BrinNaamActueel'],   # Groups
                          alpha=0.05)          # Significance level

tk = tukey.plot_simultaneous()    # Plot group confidence intervals

tukey.summary()